In [18]:
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../Data/IMDB Dataset.csv/IMDB Dataset.csv")

df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
df.iloc[0,0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [4]:
# preprocess the data
def preprocess(text):
    text = re.sub('<.*?>', ' ', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text

In [5]:
df["review"] = df["review"].apply(preprocess)

In [6]:
df.iloc[0,0]

'one of the other reviewers has mentioned that after watching just   oz episode you ll be hooked  they are right  as this is exactly what happened with me   the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word   it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away   i would say the main appeal of the show is due to the fact that it goes where other sh

In [7]:
df["sentiment"] = df["sentiment"].map({"positive":1, "negative":0})
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming t...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there s a family where a little boy ...,0
4,petter mattei s love in the time of money is...,1
...,...,...
49995,i thought this movie did a down right good job...,1
49996,bad plot bad dialogue bad acting idiotic di...,0
49997,i am a catholic taught in parochial elementary...,0
49998,i m going to have to disagree with the previou...,0


In [8]:
X = df["review"]
y = df["sentiment"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((40000,), (10000,), (40000,), (10000,))

In [20]:
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

In [21]:
bert_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(torch_device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.

In [11]:
auto_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(text):
    return auto_tokenizer(text, padding=True, truncation=True)

In [12]:
tokenized_train_dataset = X_train.map(tokenize)
tokenized_test_dataset = X_test.map(tokenize)

In [13]:
tokenized_train_dataset[0]

{'input_ids': [101, 2028, 1997, 1996, 2060, 15814, 2038, 3855, 2008, 2044, 3666, 2074, 11472, 2792, 2017, 2222, 2022, 13322, 2027, 2024, 2157, 2004, 2023, 2003, 3599, 2054, 3047, 2007, 2033, 1996, 2034, 2518, 2008, 4930, 2033, 2055, 11472, 2001, 2049, 24083, 1998, 4895, 10258, 2378, 8450, 5019, 1997, 4808, 2029, 2275, 1999, 2157, 2013, 1996, 2773, 2175, 3404, 2033, 2023, 2003, 2025, 1037, 2265, 2005, 1996, 8143, 18627, 2030, 5199, 3593, 2023, 2265, 8005, 2053, 17957, 2007, 12362, 2000, 5850, 3348, 2030, 4808, 2049, 2003, 13076, 1999, 1996, 4438, 2224, 1997, 1996, 2773, 2009, 2003, 2170, 11472, 2004, 2008, 2003, 1996, 8367, 2445, 2000, 1996, 17411, 4555, 3036, 2110, 7279, 4221, 12380, 2854, 2009, 7679, 3701, 2006, 14110, 2103, 2019, 6388, 2930, 1997, 1996, 3827, 2073, 2035, 1996, 4442, 2031, 3221, 21430, 1998, 2227, 20546, 2015, 2061, 9394, 2003, 2025, 2152, 2006, 1996, 11376, 7861, 2103, 2003, 2188, 2000, 2116, 26030, 2015, 7486, 18542, 10230, 7402, 2015, 8135, 16773, 3493, 1998, 2062,

- Training

In [25]:
training_args = TrainingArguments(
    output_dir='./results', 
    num_train_epochs=1,              
    per_device_train_batch_size=1,  
    per_device_eval_batch_size=8,   
    evaluation_strategy="steps",
    eval_steps=10,
    save_steps=10,
    warmup_steps=500,
    learning_rate=1e-3,
    logging_dir='./logs',
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = np.mean(labels == preds)
    return {
        'accuracy': acc,
    }

In [ ]:
trainer = Trainer(model=bert_model, 
                    args=training_args,
                    train_dataset=tokenized_train_dataset,
                    eval_dataset=tokenized_test_dataset,
                    compute_metrics=compute_metrics)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()